# Шпаргалка - `scikit-learn`

`scikit-learn` - это центральная библиотека машинного обучения. Существует много разных библиотек, которые содержат другие алгоритмы. Но стоит заметить, что де-факто `scikit-learn` содержит необходимый минимум, чтобы можно было работать с данными.

## Видео шаг по шагу (доп. поддержка для начинающих)

In [1]:
%%html
<iframe style="height:500px;width:100%" src="https://www.youtube.com/embed/wv7p1zuGyx4" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

Существует много библиотек, как в Python, так и в других языках программирования. Но стоит признать (и это не только мое личное мнение), что scikit-learn, действительно, ввел три стандартные строчки, которые помогли очень сильно упростить способ мышления и легко подменять модель А на модель Б.

Что это за три строчки? Сейчас увидим на примере, но сразу стоит вчитать (проимпортировать) библиотеку.

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb

from sklearn.model_selection import cross_val_score, KFold

from sklearn.metrics import mean_absolute_error

Вчитываем данные.

In [ ]:
df = pd.read_hdf("../input/train_data.h5")
print(df.shape)

## Целевая переменная
Нужно вытянуть число (цену) из `price`, которую сразу поделим на миллион, чтобы уменьшить кол-во нулей.

In [ ]:
def parse_price(x):
    return int(x.split("₽")[0].replace(" ", ""))

df["price_num"] = df["price"].map(parse_price).map(lambda x: x/1000000)

`price_num` - это наша целевая переменная, т.е. это цена за недвижимость в миллионах рублей.

Обычно целевую переменную называют `y` (с маленькой буквы, потому что - это вектор - "один столбец").


Для того, чтобы тренировать модель машинного обучения, еще нужна матрица (таблица) признаков - `X`.


В колонке `params` очень много характеристик для недвижимости. Вот как пример:

In [ ]:
df["params"].head(2).values

### Первая строка (недвижимость)
```
{
    'Охрана:': 'закрытая территория', 
    'Тип здания:': 'Монолитное', 
    'Тип объекта:': 'квартиры', 
    'Количество корпусов:': '1', 
    'Тип объявления:': 'в новостройке', 
    'Застройщик:': 'Доходный дом', 
    'Общая площадь:': '54.3 м²', 
    'Дата публикации:': '22 апреля', 
    'Количество комнат:': '2', 
    'Парковка:': 'отдельная многоуровневая, гостевая', 
    'Дата  обновления:': '18 мая', 
    'Количество этажей:': '11', 
    'Сдача:': '1 кв. 2020 года', 
    'Комиссия агенту:': 'без комиссии', 
    'Высота потолков:': '2.8 м', 
    'Этаж:': '8/11', 
    'Этап строительства:': 
    'Возведение стен', 'Новостройка:': 
    'Апарт-комплекс Nord, м. Алтуфьево', 
    'Количество квартир:': '163', 
    'Класс жилья:': 'Комфорт класс', 
    'Адрес:': ''}
```


### Вторая строка (недвижимость)
```
{
    'Лифт:': 'да', 
    'Тип здания:': 'Монолитное', 
    'Тип объекта:': 'квартира', 
    'Количество корпусов:': '5', 
    'Тип объявления:': 'в новостройке', 
    'Застройщик:': 'MR Group', 
    'Общая площадь:': '38 м²', 
    'Вид из окна:': 'на улицу', 
    'Количество комнат:': '2', 
    'Парковка:': 'подземная', 
    'Дата  обновления:': '16 мая', 
    'Класс жилья:': 'Комфорт класс', 
    'Дата публикации:': '10 мая', 
    'Количество этажей:': '31', 
    'Сдача:': '2 кв. 2021 года', 
    'Комиссия агенту:': 'без комиссии', 
    'Высота потолков:': '3.15 м', 
    'Этаж:': '6/31', 
    'Этап строительства:': 'Возведение стен', 
    'Новостройка:': 'ЖК Discovery (ЖК «Дискавери»), м. Ховрино', 
    'Количество квартир:': '1251', 
    'Возможна ипотека:': 'да', 
    'Адрес:': ''}
```


Обрати внимание, что каждая недвижимость описана при помощи различных характеристик. Например, информация о том, есть ли лифт или тип здания. Таке можешь заметить, что разная недвижимость имеет общие характеристики, но не всегда те же самые.

Кстати, эти характеристики в машинном обучении называются признакамии (анг. `features`). Важно, чтобы в итоге каждый объект был описан теми самыми признаками. А как же это сделать, если у нас такая инфомрация не всегда имеется? Если в оригинальных данных нет какого-то значения - тогда там будет на называемый `NaN` (т.е. отсутсвие значения).

Давай вытянем все значения и добавим их к `params`.

In [ ]:
params = df["params"].apply(pd.Series)
print(params.shape)
params

Получается, что у нас есть до 55 характеристик (признаков) для каждой недвижимости. Не всегда все 55 признаков имеют значения - там где есть пропуски, мы видим значения `NaN`.

Для начала можем заполнить просто значением -1. Почему так? Чтобы не было пропусков.

In [ ]:
params = params.fillna(-1)
params

Для того, чтобы начать тренировать модель, нужно приготовить данные, чтобы там были только числа. Поэтому давай сейчас сделаем такой трик, что вытянуть данные типа `object` (обычно это строки), используя функцию `.factorize()`.

In [ ]:
obj_feats = params.select_dtypes(object).columns
obj_feats

Вот все эти столбцы типа `object`. Запускаем теперь `.factorize()` и результат записываем в новую переменную с префиксом `_cat`.

In [ ]:
for feat in obj_feats:
    df["{}_cat".format(feat)] = params[feat].factorize()[0]
    

cat_feats = [x for x in df.columns if "_cat" in x]
cat_feats

Все это пусть будет нашими признаками и, таким образом, можем создать матрицу (таблицу) признаков - X.

In [ ]:
X = df[cat_feats]
y = df["price_num"]

X.shape, y.shape

Напомню еще раз `X` - это таблица, поэтому пишем с большой буквы, а `y` - пишем с маленькой буквы, потому что это вектор (один столбец из таблицы).


Важно, чтобы количество строк было одинаковым: 22 732.


## Тренируем модель
Три "главные" строчки из scikit-learn ;).

In [ ]:
model = DummyRegressor()
model.fit(X, y)
y_pred = model.predict(X)

В первой строчке мы выбираем, какой будем использовать алгоритм. 
Во второй строчке тренируем модель - `.fit(X, y)`. Чтобы тренировать модель нам нужен `X` (таблица признаков) и `y` (целевая функция).
В третьей строчке, когда модель уже обучена - делаем прогнозирование и результат (прогнозирования) записываем в переменную `y_pred`.

Если мы решились поменять модель А на модель Б, то нужно будет поменять только первую строчку и все. Но, возможно, Тебе приходилось увидеть различные кошмары, так называемого кода "спагетти", где для того, чтобы начать использовать другой алгоритм, обычно нужно очень сильно помучаться.

Часто можно сделать так, что мы итерируем различные модели автоматически. Обычно это имеется в виду так называемый  `automl`. Хотя концептуально `automl`, значит полностью автоматизированное машинное обучение. На практие не все так просто и только некоторые шаги во всем процессе машинного обучения можно легко автоматизировать. Кстати, этот процесc перебора моделей, называется - выбор модели (анг. `model selection`).


А насколько хороший у нас результат? Для этого обычно используют метрики успеха.
Существуют различные метрики. Одна из них `mean_asbolute_error`.

In [ ]:
mean_absolute_error(y, y_pred)

Для упрощения, можно сказать, что это плюс/минус. Получается, мы ошибаемся на данный момент плюс/минус на 13 млн. рублей. Много это или мало? В данном случае в среднем цена недвижимостти 17 млн. рублей.

In [ ]:
df["price_num"].mean()

Поэтому ошибиться на 13 млн, когда в среднем цена 17млн. - это достаточно грубо. Но стоит заметить, что на данный момент - это очень простая модель. Давай выберем чуть труднее модель - дерево решений.

In [ ]:
model = DecisionTreeRegressor(max_depth=7)
model.fit(X, y)
y_pred = model.predict(X)

mean_absolute_error(y, y_pred)

Как видишь, в коде поменялась только одна строчка - первая: `model = DecisionTreeRegressor(max_depth=7)`. Все остальное без изменений.

Видно, что качество стало лучше.


Теперь можем попробовать еще более сложную модель `xgboost`. Что интересно, `xgboost` - это библиотека, написанная на С++ (это другой язык программирования). В данном случае Python - это только интерфейс к этой библиотеке. Интересно также, что этот интерфейс был так написан, чтобы он подходил под стандарты scikit-learn (хотя это было не сразу). Но благодаря этому, опять заменяем только одну строку (первую) и тренируем новую модель (в данном случае `xgboost`).

In [ ]:
model = xgb.XGBRegressor(max_depth=7, n_estimators=50, random_state=0)
model.fit(X, y)
y_pred = model.predict(X)

mean_absolute_error(y, y_pred)

Получаем еще лучше результат, около 2.5 млн рублей. Но, рано радоваться. Ведь в данном случае, скорее всего, модель переобучилась. 
Обрати внимание, что мы тренируем и прогнозируем модель на тех же самых данных (тот же самый `X`). И как результат - модель вместо того, чтобы обобщить знания - могла просто запомнить конкретные случаи.


Чтобы бороться с переобучением, существуют разные способы. Например, валидация (на практике часто применяется кросс-валидация). Самый простой способ - применить `.cross_val_score()`.

In [ ]:
model = xgb.XGBRegressor(max_depth=7, n_estimators=50, random_state=0)
scores = cross_val_score(model, X, y, scoring="neg_mean_absolute_error")
np.mean(scores), np.std(scores)

В данном случае видно, что качество около 5млн рублей.


## Кросс-валидация
А как это работает?

1. Берем все наши данные (все строчки) - 100%
2. Решаем - на сколько частей хотим поделить данные (кстати, эти части называются фолдами - анг. `fold`), например, может быть 5 или 10 частей
3. Делим наши данные на количество частей (шаг 2) 
4. Дале одна часть попадает в тест, а все остальные для тренировки
5. Каждая часть один раз будет в тесте.

Давай посмотрим на примере.
1. Представь, что у нас 100 строк.
2. Мы решаем, что у нас будет 4 части (фолда).
3. Делим наши данные на 4 частии (1-25, 26-50, 51-75, 76-100).

Так как мы выбрали 4 части - это значит мы будем тренировать 4 модели. Каждая часть - один раз.

```
model = ...
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
```

#### Итерация 1
- `X_train`, `y_train` => строчки от 26 до 100.
- `X_test`, `y_test` => строчки 1-25


#### Итерация 2
- `X_train`, `y_train` => строчки от 1 до 25 и от 51 до 100.
- `X_test`, `y_test` => строчки 26-50

#### Итерация 3
- `X_train`, `y_train` => строчки от 1 до 50 и от 76 до 100.
- `X_test`, `y_test` => строчки 51-75

#### Итерация 4
- `X_train`, `y_train` => строчки от 1 до 75.
- `X_test`, `y_test` => строчки 76-100


    Используем функцию `cross_val_score()`, в которой есть параметр `cv`, для данного параметра указываем значение, например, 4 - `cv=4`, что означает, что будет 4 части (фолда), т.е. пример, который мы разобрали выше.


Вот еще визуально, то же самое - чтобы закрепить материал (здесь было 5 частей - фолдов). Плюс там еще более сложная структура. Ну это уже нюансы, с которыми стоит разобраться на следующем шаге.
![](../images/cv.png)
*Рисунок взят с сайтa [scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html)*


Функция  `cross_val_score` обычно очень удобная, но этого не всегда достаточно. Тогда можно код расписать на более мелкие шаги.

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)
scores = []
for train_idx, test_idx in tqdm(cv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = xgb.XGBRegressor(max_depth=7, n_estimators=50, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = mean_absolute_error(y_test, y_pred)
    scores.append(score)
    
np.mean(scores), np.std(scores)

А зачем нам это? Ну, например, мы можем сделать такой трюк. Чтобы "немного" улучшить ситуацию с отстающими значениями - можем эти значения логарифмировать. Далее обучать данные уже на логарифмированных ценах. Но потом вернуть обратно, в изначальные цены.

Для того, чтобы найти логарифм, используем функцию `np.log()`. Обратная функция - `np.exp()`.

In [ ]:
np.log(10)

Теперь пробуем вернуть назад. Для этого используем функцию `np.exp()`

In [ ]:
np.exp(np.log(10))

Видишь, получилось 10 (и длинный хвост), но это не так важно - это мелочи 😉 или кошмары цифрового мира 😱.

Теперь логарифмируем наши цены - это звучит может "заумно", но идея очень простая.

In [ ]:
y_log = np.log(y)

Все!  У нас уже есть цена, пропущенная через логарифм. Теперь `y_log` будет обучать нашу модель. Иными словам, наша модель никогда не увидит настоящие цены,а значит тоже будет прогнозировать логарифмированные ценны, т.е. будет возвращать `y_log_pred`.  Для того, чтобы вернуть в обычные цены, пропускаем через `np.exp`.

`y_pred = np.exp(y_log_pred)`

А теперь все вышеописанное собираем вместе.

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)
scores = []
for train_idx, test_idx in tqdm(cv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_log_train, y_test = y_log.iloc[train_idx], y.iloc[test_idx]

    model = xgb.XGBRegressor(max_depth=7, n_estimators=50, random_state=0)
    model.fit(X_train, y_log_train)
    y_log_pred = model.predict(X_test)
    y_pred = np.exp(y_log_pred)
    
    score = mean_absolute_error(y_test, y_pred)
    scores.append(score)
    
np.mean(scores), np.std(scores)

Видишь, такое простое изменение (обучение на ценах, пропущенных через логарифм) и модель улучшилась, используя такой простой трюк.

Почему это работает? Средняя цена лучше себя ведет после логарифмирования. Смотри -  средняя цена и медиана будут почти одинаковыми (в отличие от оригинальной цены, где средняя "очень искажена").

In [ ]:
y_log.mean(), y_log.median()

Можно еще посмотреть на визуализацию и увидеть, что данные распределены симметрично (видна середина, где находится большинство объектов).

In [ ]:
y_log.hist(bins=100);

Почему алгоритм лучше работает при симметричном распределении? Для упрощения можешь запомнить одну простую вещь, что когда распределение симметрично, то тогда это обычно работает лучше (использую статистические методы). Это такое правило как "для детей", но для начала этого достаточно - потом можно копать глубже.